## Import Libraries

In [160]:
import pandas as pd
import numpy as np
import random

SEED = 42

## Load and view data

In [161]:
path = 'data/ai-medical-chatbot-data.csv'
df = pd.read_csv(path)

In [162]:
df.sample(5, random_state=SEED)

,short_question,short_answer,tags,label
30265,what is a nervous breakdown,nervous breakdown is not a clinical term so it does not mean anything specific to doctors people may use nervous breakdown to describe an episode of severe emotional distress the word “breakdown” may suggest the person showed unusual behavior or stopped performing his or her expected duties at home or work people described as having a nervous breakdown may or may not meet the criteria for a formal psychiatric diagnosis,['mental breakdown'],1.0
24640,i get very paranoid about allergies and death please help,do not be so obsessive about it in last 20 years i have hardly seen any case of anaphylaxis due to allergy its that rare so calm down you wont get anaphylaxis,['paranoia'],1.0
13125,my husband took a 25 mg imitrex for a really bad headache over three hours ago can he also take vicodin,hi start with the ringing well that could be tinnitus have a look on www earthclnic com its all natural stuff you may find some help there if you think you ears are clogged try some warm olive oil in them its worked for me a couple of times as fore smoking did smoke form the age of 11 till 1996 have just resently had a bit of tinnitus but i put this down to using a cutting disc next to my head as i was up a ladder your find tinnitus comes from lots of things load music in ear phones being the worst do hope this helps you good luck,['headache' 'vitamin a'],-1.0
41494,my wife is 72 and was diagnosed with afib 3 years ago is it advisable for her to be on a statin medication now her health has been good in general except for age related issues like arthritis and some hearing loss she has never had any heart problems before the afib never any high blood pressure high cholesterol or anything else her last cholesterol level was 186 triglyceride 128 hdl 68 non hdl cholesterol 118 and ldl 92 i would like to know what the recommendation is for a person with this profile should they be taking a statin medication as a prophylactic measure,hi her numbers are good and keep as far away from statins as she can get she sounds really healthy statins could reverse that if your not up on statins just go to www spacdoc com its a site all by doctors even the site owner is a doctor makes for some very interesting reading ok good luck,['arthritis' 'hearing loss' 'heart' 'high blood pressure' 'cholesterol'],1.0
29503,im having lower back pains and im having pains in my pelvic area and i am leaking urine for no reason i have to use the restroom about every hour i feel sick every time i wake up i have no idea whats going on i have never had a kid before i dont know what kind of doctor i need to see,mupiricin is an antibiotic used for skin infections it will not help poison ivy sorry,['pelvic area' 'nausea' 'vision' 'pain' 'back pain'],-1.0


#### Convert the tags column from string to list

In [163]:
def convert_col_to_list(df : pd.DataFrame, column_name : str) -> pd.DataFrame:
	
	# Iterate through each row in the DataFrame
	for i in range(len(df)):  
		value = df.at[i, column_name]
		
		# Remove square brackets from the string (if present)
		value_without_brackets = value.replace('[', '').replace(']', '')
		
		# Split the cleaned string into a list by spaces
		value_as_list = value_without_brackets.split("' '")

		# Remove any remaining single quotes from the start and end
		value_as_list[0] = value_as_list[0].lstrip("'")
		value_as_list[-1] = value_as_list[-1].rstrip("'")

		# Remove commas from values
		cleaned_value_as_list = [val.replace(',', '') for val in value_as_list]
		
		df.at[i, column_name] = cleaned_value_as_list
	
	return df


df = convert_col_to_list(df, 'tags')

In [164]:
df.sample(5, random_state=SEED)

,short_question,short_answer,tags,label
30265,what is a nervous breakdown,nervous breakdown is not a clinical term so it does not mean anything specific to doctors people may use nervous breakdown to describe an episode of severe emotional distress the word “breakdown” may suggest the person showed unusual behavior or stopped performing his or her expected duties at home or work people described as having a nervous breakdown may or may not meet the criteria for a formal psychiatric diagnosis,[mental breakdown],1.0
24640,i get very paranoid about allergies and death please help,do not be so obsessive about it in last 20 years i have hardly seen any case of anaphylaxis due to allergy its that rare so calm down you wont get anaphylaxis,[paranoia],1.0
13125,my husband took a 25 mg imitrex for a really bad headache over three hours ago can he also take vicodin,hi start with the ringing well that could be tinnitus have a look on www earthclnic com its all natural stuff you may find some help there if you think you ears are clogged try some warm olive oil in them its worked for me a couple of times as fore smoking did smoke form the age of 11 till 1996 have just resently had a bit of tinnitus but i put this down to using a cutting disc next to my head as i was up a ladder your find tinnitus comes from lots of things load music in ear phones being the worst do hope this helps you good luck,"[headache, vitamin a]",-1.0
41494,my wife is 72 and was diagnosed with afib 3 years ago is it advisable for her to be on a statin medication now her health has been good in general except for age related issues like arthritis and some hearing loss she has never had any heart problems before the afib never any high blood pressure high cholesterol or anything else her last cholesterol level was 186 triglyceride 128 hdl 68 non hdl cholesterol 118 and ldl 92 i would like to know what the recommendation is for a person with this profile should they be taking a statin medication as a prophylactic measure,hi her numbers are good and keep as far away from statins as she can get she sounds really healthy statins could reverse that if your not up on statins just go to www spacdoc com its a site all by doctors even the site owner is a doctor makes for some very interesting reading ok good luck,"[arthritis, hearing loss, heart, high blood pressure, cholesterol]",1.0
29503,im having lower back pains and im having pains in my pelvic area and i am leaking urine for no reason i have to use the restroom about every hour i feel sick every time i wake up i have no idea whats going on i have never had a kid before i dont know what kind of doctor i need to see,mupiricin is an antibiotic used for skin infections it will not help poison ivy sorry,"[pelvic area, nausea, vision, pain, back pain]",-1.0
